# Measuring Consistency

In [0]:
#importing packages
import numpy as np
import torch
import os
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import cv2
from PIL import Image

import os
import random
import shutil
import time
import warnings
import sys
import numpy as np
import os
import pandas as pd
import h5py
import logging
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from google.colab import drive
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import DataLoader


In [0]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/drive/My Drive/Thesis re-id/triplet-reid-master")

It is important to note that for measuring consistency, the market1501_shift.py file is used. This is due to the fact that this experiment does not use the test_embeddings data augmentation existing of TenCrops.

In [0]:
from own_code.loss import TripletLoss
from own_code.triplet_selector import BatchHardTripletSelector
from own_code.batch_sampler import BatchSampler
from own_code.Market1501_shift import Market1501
from own_code.backbone import EmbedNetwork
#from own_code.Market1501_no_aug import Market1501
from own_code.optimizer import AdamOptimWrapper
from logger import logger
from models_lpf import *
#import models_lpf.resnet
import models_lpf.resnet
from utils import pdist_np as pdist


In [0]:
# Labels
labels = pd.read_csv('data/market1501_train.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_query = pd.read_csv('data/market1501_query.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_test = pd.read_csv('data/market1501_test.csv', names = ['pid', 'fid'], header = None, dtype = str)

### Reading the h5py Files

In [0]:
fileName = 'data_final.h5'
fileName_qt = 'data_qt.h5'

In [0]:
with h5py.File(fileName, "a") as out:
  X_train = np.asarray(out["X_train"])
  #Y_train = np.asarray(out["Y_train"])
  

In [0]:
with h5py.File(fileName_qt, "a") as out:
  X_query = np.asarray(out["X_dev"])
  X_test = np.asarray(out["X_test"])

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [0]:
X_query.shape

(3368, 128, 64, 3)

In [0]:
sum(X_query).shape

(128, 64, 3)

## Loading model

## Validate_shift function 

In [0]:
def validate_shift(fids, net, dataset, model_num):
  batch_time = AverageMeter()
  consist = AverageMeter()
  torch.multiprocessing.set_sharing_strategy('file_system')

  ## logging
  FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
  logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
  logger = logging.getLogger(__name__)
  ## restore model
  logger.info('restoring model')
  model = net
  #model = nn.DataParallel(model)
  model = net.cuda()
  model.module.load_state_dict(torch.load('./res/model{}.pkl'.format(model_num)))
  model = nn.DataParallel(model)
  model.eval()
  batchsize = 32
  ds = Market1501(pids_list=list(fids), array=dataset, is_train = False)
  dl = DataLoader(ds, batch_size = batchsize, drop_last = False, num_workers = 4)
  
  # Loading gallery embeddings
  test_embs = "./res/emb_test{}.pkl".format(model_num)
  with open(test_embs, 'rb') as fr:
        gallery_dict = pickle.load(fr)
        emb_gallery = gallery_dict['embeddings']

  ## embedding samples
  logger.info('start embedding')
  all_iter_nums = len(ds) // batchsize + 1
  embeddings = []
  label_ids = []
  label_cams = []

  epochs_shift = 100
  print("embeddings are loaded from gallery embeddings created with{}".format(model_num))
  with torch.no_grad():
    end = time.time()
    for ep in range(epochs_shift):
      for it, (img, lb_id, lb_cam) in enumerate(dl):
        label_ids.append(lb_id)
        label_cams.append(lb_cam)
        embds = []
        
        off0a = np.random.randint(32,size=2)
        off0b = np.random.randint(16,size=2)
        off1a = np.random.randint(32,size=2)
        off1b = np.random.randint(16,size=2)
        img = img.cuda()
        output0 = model(img[:,:,off0a[0]:off0a[0]+224,off0b[0]:off0b[0]+112])
        output1 = model(img[:,:,off1a[0]:off1a[0]+224,off1b[0]:off1b[0]+112])
        output0 = output0.cpu().numpy()
        output1 = output1.cpu().numpy()

        ## calculate distances with output0 and output1
        out0_mtx = pdist(output0, emb_gallery)
        out1_mtx = pdist(output1, emb_gallery)
        out0_mtx = torch.from_numpy(out0_mtx)
        out1_mtx = torch.from_numpy(out1_mtx)

        # agreement
        pred0 = out0_mtx.argmin(axis=1)
        pred1 = out1_mtx.argmin(axis=1)
        agree = pred0.eq(pred1)
        agree = 100.*torch.mean(agree.type(torch.FloatTensor).to(torch.device("cuda")))
        consist.update(agree.item(), img.size(0))
        
        if it % 30 == 0:
          print('Ep [{0}/{1}]:\t'
          'Test: [{2}/{3}]\t'
          'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
          'Consist {consist.val:.4f} ({consist.avg:.4f})\t'.format(
           ep, epochs_shift, it, len(dl), batch_time=batch_time,
            consist=consist))

      print(' * Consistency {consist.avg:.3f}'
            .format(consist=consist))
  
  return consist.avg

## Consistency model 1


In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

NameError: ignored

In [0]:
model_num = 1
logger.info('setting up backbone model and loss')
net = EmbedNetwork(pretrained_base=True).cuda()
net_1 = nn.DataParallel(net)
summary(net_1, (3,128,64))

setting up backbone model and loss


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 32, 16]               0
            Conv2d-5           [-1, 64, 32, 16]           4,096
       BatchNorm2d-6           [-1, 64, 32, 16]             128
              ReLU-7           [-1, 64, 32, 16]               0
            Conv2d-8           [-1, 64, 32, 16]          36,864
       BatchNorm2d-9           [-1, 64, 32, 16]             128
             ReLU-10           [-1, 64, 32, 16]               0
           Conv2d-11          [-1, 256, 32, 16]          16,384
      BatchNorm2d-12          [-1, 256, 32, 16]             512
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
consistency = validate_shift(fids = labels_query.fid, net = net_1, dataset = X_query, model_num = model_num)

embeddings are loaded from gallery embeddings created with1
Ep [0/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 81.2500 (81.2500)	
Ep [0/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 81.2500 (82.7621)	
Ep [0/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 71.8750 (84.8873)	
Ep [0/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 81.2500 (84.0659)	
 * Consistency 84.234
Ep [1/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 78.1250 (84.1765)	
Ep [1/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 81.2500 (84.1055)	
Ep [1/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 68.7500 (84.3233)	
Ep [1/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 81.2500 (84.4745)	
 * Consistency 84.516
Ep [2/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 75.0000 (84.4710)	
Ep [2/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 71.8750 (84.3168)	
Ep [2/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 75.0000 (84.0470)	
Ep [2/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 81.2500 (84.1211)	
 * Consistency 84.175
E

## Consistency model 6

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 6
filter_size = 3
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embed = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embed = embed

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embed(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_6 = nn.DataParallel(model)
summary(net_6, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 65, 33]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
consistency = validate_shift(fids = labels_query.fid, net = net_6, dataset = X_query, model_num = model_num)

embeddings are loaded from gallery embeddings created with6
Ep [0/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 87.5000 (87.5000)	
Ep [0/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 90.6250 (85.4839)	
Ep [0/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 84.3750 (86.5779)	
Ep [0/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 84.3750 (87.6030)	
 * Consistency 87.678
Ep [1/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 87.5000 (87.6765)	
Ep [1/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 100.0000 (87.5459)	
Ep [1/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 81.2500 (87.6316)	
Ep [1/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 75.0000 (87.5000)	
 * Consistency 87.381
Ep [2/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 90.6250 (87.3966)	
Ep [2/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 93.7500 (87.0083)	
Ep [2/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 90.6250 (86.9936)	
Ep [2/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 90.6250 (87.0129)	
 * Consistency 87.074


## Consistency model 7

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 7

In [0]:
filter_size = 2
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_7 = nn.DataParallel(model)
summary(net_7, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 64, 32]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
consistency = validate_shift(fids = labels_query.fid, net = net_7, dataset = X_query, model_num = model_num)

embeddings are loaded from gallery embeddings created with7
Ep [0/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 84.3750 (84.3750)	
Ep [0/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 93.7500 (84.4758)	
Ep [0/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 90.6250 (86.1680)	
Ep [0/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 78.1250 (86.4011)	
 * Consistency 86.283
Ep [1/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 84.3750 (86.2647)	
Ep [1/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 90.6250 (86.1009)	
Ep [1/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 90.6250 (86.1278)	
Ep [1/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 84.3750 (85.9713)	
 * Consistency 85.986
Ep [2/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 81.2500 (85.9634)	
Ep [2/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 90.6250 (85.5202)	
Ep [2/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 96.8750 (85.5433)	
Ep [2/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 78.1250 (85.2508)	
 * Consistency 85.253
E

## Model 8 consistency

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 8

In [0]:
filter_size = 5
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_8 = nn.DataParallel(model)
summary(net_8, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 67, 35]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
consistency = validate_shift(fids = labels_query.fid, net = net_8, dataset = X_query, model_num = model_num)

embeddings are loaded from gallery embeddings created with8
Ep [0/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 93.7500 (93.7500)	
Ep [0/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 87.5000 (86.3911)	
Ep [0/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 81.2500 (87.7561)	
Ep [0/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 90.6250 (88.0838)	
 * Consistency 88.302
Ep [1/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 100.0000 (88.4118)	
Ep [1/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 81.2500 (87.9817)	
Ep [1/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 78.1250 (88.1203)	
Ep [1/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 96.8750 (88.0573)	
 * Consistency 88.153
Ep [2/100]:	Test: [0/106]	Time 0.000 (0.000)	Consist 96.8750 (88.1944)	
Ep [2/100]:	Test: [30/106]	Time 0.000 (0.000)	Consist 90.6250 (88.2117)	
Ep [2/100]:	Test: [60/106]	Time 0.000 (0.000)	Consist 96.8750 (88.1215)	
Ep [2/100]:	Test: [90/106]	Time 0.000 (0.000)	Consist 68.7500 (88.1012)	
 * Consistency 88.213
